# Collect list of files with CCTP content and EBP reference

In [120]:
import pandas as pd
import numpy as np
import re
import os # This module is great for path manipulation

from sklearn.model_selection import cross_val_score
from skrub import TableReport
from skrub import tabular_learner

#file_script_result = r"C:\Users\jch_m\DocumentsPerso\Centrale-IAC\StageATAE\listfile_FAMARO2.txt"
#file_script_result = r"C:\Users\jch_m\DocumentsPerso\Centrale-IAC\StageATAE\result_scan_AO.txt"
file_script_result = r"C:\Users\jch_m\DocumentsPerso\Centrale-IAC\StageATAE\result_scan_AO\result_scan_AO_v2.txt"


Extraction du contenu du log de script scannant les repertoires onedrive des CSPS
- num commande EBP => construction df_EBP
- filename de CCTP => construction df_consult

In [139]:
# init variables
EBP = False
CCTP = False
nb_valid_EBP = 0
nb_EBP_file = 0
nb_valid_CCTP = 0
nb_CCTP_file = 0
consult_non_valide = 0
df_EBP = pd.DataFrame()
df_consult = pd.DataFrame()

# pour chaque ligne du fichier
with open(file_script_result, 'r', encoding='utf-16') as file:
    print(f"Contenu de '{file_script_result}':")
    for line_number, line in enumerate(file, 1):
        cur_line = line.strip()

        # reinit variables
        numero_commande = ""
        ebp_status = "unknown"

        if "=== Liste ref EBP ===" in line:
            print("Processing des EBP")
            EBP = True
            CCTP = False
        elif "=== Liste fichiers CCTP ===" in cur_line:
            print("total EBP file:", nb_EBP_file, " Valid EBP:", nb_valid_EBP, "nb line read:",line_number)
            line_num_ebp = line_number
            print("Processing des CCTP")
            EBP = False
            CCTP = True
        else:
            # Extraction info du path

            ##### CSPS Name extraction
            parts = line.split("\\ATAE\\") 
            #print(len(parts), line)
            if len(parts) > 1:
                remaining_path = parts[1]
                first_backslash_index = remaining_path.find('\\')
                #print(remaining_path, first_backslash_index)
                if first_backslash_index != -1:
                    extracted_name = remaining_path[:first_backslash_index]
                    #print(f"Nom extrait : {extracted_name}")
                else:
                    extracted_name = "Not found (no backslash after ATAE)"
                    continue
            else:
                extracted_name = "Not found (ATAE not in path)"
                continue

            ##### Full File Path extraction
            directory_path = os.path.dirname(cur_line)
            parts_for_folder = cur_line.split(extracted_name, 1) # Split by the extracted name, once
            if len(parts_for_folder) > 1:
                path_after_name = parts_for_folder[1] # This starts with the first backslash after the name
                path_before_filename = os.path.dirname(cur_line)
                name_end_pos = cur_line.find(extracted_name) + len(extracted_name)
                
                escaped_extracted_name = re.escape(extracted_name)   
                idx_name = cur_line.find(extracted_name)
                
                idx_filename_start = cur_line.rfind(os.sep) # Last separator

                if idx_name != -1 and idx_filename_start != -1:
                    start_index_segment = idx_name + len(extracted_name) + 1
                    end_index_segment = idx_filename_start + 1 # Include the trailing backslash

                    if start_index_segment < end_index_segment:
                        directory_name = cur_line[start_index_segment:end_index_segment]
                        
                        # EBP status en fonction du path
                        if "00-DEVIS" in directory_name or "00- DEVIS" in directory_name or "02-DEVIS A" in directory_name:
                            if "annul" in directory_name.lower():
                                ebp_status = "Annule"
                            elif "echec" in directory_name.lower():
                                ebp_status = "Perdu"
                            elif "non repondu" in directory_name.lower():
                                ebp_status = "Non repondu"
                            else:
                                ebp_status = "Offre en cours de decision"
                        elif "01-CHANTIERS" in directory_name or "01- CHANTIERS" in directory_name or "01 - CHANTIERS" in directory_name:
                            if "annul" in directory_name.lower():
                                ebp_status = "Annule"
                            elif "echec" in directory_name.lower():
                                ebp_status = "Perdu"
                            elif "non repondu" in directory_name.lower():
                                ebp_status = "Non repondu"
                            else:
                                ebp_status = "Chantiers"
                        elif "ARCHIVE" in directory_name:
                            if "annul" in directory_name.lower():
                                ebp_status = "Annule"
                            elif "echec" in directory_name.lower():
                                ebp_status = "Perdu"
                            elif "non repondu" in directory_name.lower():
                                ebp_status = "Non repondu"
                            else:
                                ebp_status = "Archive"
                    else:
                        directory_name = "Segment invalide"
                else:
                    directory_name = "Segment invalide"
                #print(f"Chemin du dossier extrait : {directory_name}")

                ##### Filename extraction
                file_name = os.path.basename(cur_line)
                #print(f"Nom du fichier : {file_name}")

            if EBP and "invalide" not in directory_name :
                if "de ebp " in line.lower():
                    nb_EBP_file += 1
                    match_numero = re.search(r"DE EBP (\d+)", cur_line, re.IGNORECASE)
                    numero_commande = match_numero.group(1) if match_numero else "Non trouvé"
                
                    #before_commande_ebp = cur_line.lower().split("de ebp", )[0]
                    before_commande_ebp = cur_line.split("DE EBP")[0]
                    #print(before_commande_ebp)
                    #match_details = re.search(r"\\([^\\]+)-([^\\]+)-([^\\]+)$", before_commande_ebp)
                    match_details = re.search(r"\\([^\\]+) - ([^\\]+) -(.*)\\([^\\]+)$", before_commande_ebp)
                    
                    if match_details:
                        ville_code_dep = match_details.group(1)
                        entreprise = match_details.group(2)
                        mission_name = match_details.group(3)
                        
                        #print(f"Commande:{numero_commande} Ville:{ville_code_dep} Entreprise:{entreprise} Mission:{mission_name}")
                        nb_valid_EBP += 1

                        # Create a new row as a dictionary
                        new_row_EBP = {
                            'SPS Name': extracted_name,
                            'Directory': directory_name,
                            'ID EBP': numero_commande,
                            'Ville': ville_code_dep,
                            'Entreprise': entreprise,
                            'Mission': mission_name,
                            'statut': ebp_status,
                        }
                        df_EBP = pd.concat([df_EBP, pd.DataFrame([new_row_EBP])], ignore_index=True)
                    else:
                        #print("Détails EBP (ville, entreprise, rénovation) : Non trouvés")
                        pass

            if CCTP and "invalide" not in directory_name:
                if "consult" in cur_line.lower():
                    #print(cur_line)
                    nb_CCTP_file += 1
                    match = re.search(r"\\consult", line, re.IGNORECASE)
                    if match:
                        before_consult = line[:match.start()]
                        match_details = re.search(r"\\([^\\]+)-([^\\]+)-(.*)\\([^\\]+)$", before_consult)

                        if match_details:
                            ville_code_dep = match_details.group(1)
                            entreprise = match_details.group(2)
                            mission_name = match_details.group(3)
                            #print(f"Ville:{ville_code_dep} Entreprise:{entreprise} Mission:{mission_name}")
                            nb_valid_CCTP += 1

                            new_row_consult = {
                            'SPS Name': extracted_name,
                            'Directory': directory_name,
                            'ID EBP': numero_commande,
                            'Ville': ville_code_dep,
                            'Entreprise': entreprise,
                            'Mission': mission_name,
                            'statut': ebp_status,
                            'filename': file_name,
                            }
                            df_consult = pd.concat([df_consult, pd.DataFrame([new_row_consult])], ignore_index=True)
                        else:
                            consult_non_valide += 1
                            #print("Détails Consultation (ville, entreprise, rénovation) : Non trouvés", before_consult)
                            pass

print("total CCTP file:", nb_CCTP_file, " Valid CCTP:", nb_valid_CCTP, " non valide:",consult_non_valide," nb lines read:",line_number-line_num_ebp)

Contenu de 'C:\Users\jch_m\DocumentsPerso\Centrale-IAC\StageATAE\result_scan_AO\result_scan_AO_v2.txt':
Processing des EBP
total EBP file: 3235  Valid EBP: 2961 nb line read: 3238
Processing des CCTP
total CCTP file: 4843  Valid CCTP: 1897  non valide: 1895  nb lines read: 133429


In [127]:
print("total EBP file:", nb_EBP_file, " Valid EBP:", nb_valid_EBP)
print("total CCTP file:", nb_CCTP_file, " Valid CCTP:", nb_valid_CCTP)

df_EBP.info()

total EBP file: 3235  Valid EBP: 2961
total CCTP file: 4843  Valid CCTP: 1897
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2961 entries, 0 to 2960
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   SPS Name    2961 non-null   object
 1   Directory   2961 non-null   object
 2   ID EBP      2961 non-null   object
 3   Ville       2961 non-null   object
 4   Entreprise  2961 non-null   object
 5   Mission     2961 non-null   object
 6   statut      2961 non-null   object
dtypes: object(7)
memory usage: 162.1+ KB


In [140]:
TableReport(df_EBP)


Processing column   7 / 7


,,,,,,,


In [147]:
pd.set_option('display.width', 500)
filtered_ebp = df_EBP[
    (df_EBP['SPS Name'] == "Cyrille CHARTIER - CYRILLE") &
    (df_EBP['statut'] == "Chantiers") &
    (df_EBP['Entreprise'] == "MAIRIE")  
].head(10)

print(filtered_ebp)

                      SPS Name                                          Directory  ID EBP            Ville Entreprise                                         Mission     statut
1   Cyrille CHARTIER - CYRILLE  01-CHANTIERS\00-NOUVEAU\CHAUMES EN RETZ - MAIR...  250492  CHAUMES EN RETZ     MAIRIE                          ancien bureau de poste  Chantiers
2   Cyrille CHARTIER - CYRILLE  01-CHANTIERS\00-NOUVEAU\DONGES - MAIRIE - Réno...  250376           DONGES     MAIRIE             Rénov périscolaire et extrascolaire  Chantiers
6   Cyrille CHARTIER - CYRILLE  01-CHANTIERS\00-NOUVEAU\MONTAGNE - MAIRIE - Ce...  230873         MONTAGNE     MAIRIE   Centre technique municipal(En attente budget)  Chantiers
7   Cyrille CHARTIER - CYRILLE  01-CHANTIERS\00-NOUVEAU\PORNIC - MAIRIE - Remp...  250425           PORNIC     MAIRIE                         Remplacement couverture  Chantiers
10  Cyrille CHARTIER - CYRILLE  01-CHANTIERS\00-NOUVEAU\PORNICHET - MAIRIE - G...  241142        PORNICHET     MAIR

In [149]:
filtered_consult = df_consult[
    (df_consult['SPS Name'] == "Cyrille CHARTIER - CYRILLE") &
    (df_consult['statut'] == "Chantiers") &
    (df_consult['Entreprise'] == "MAIRIE")  
].head(10)

print(filtered_consult)

Empty DataFrame
Columns: [SPS Name, Directory, ID EBP, Ville, Entreprise, Mission, statut, filename]
Index: []


In [133]:
TableReport(df_consult)

Processing column   8 / 8


,,,,,,,,


In [134]:

df_chantier = df_consult[df_consult['Directory'].str.contains("CHANTIER", case=False, na=False)]
df_chantier.head()


,SPS Name,Directory,ID EBP,Ville,Entreprise,Mission,statut,filename
0,Cyrille CHARTIER - CYRILLE,01-CHANTIERS\00-NOUVEAU\BERNERIE - LAD - PRE B...,,BERNERIE,LAD,PRE BOISMAIN,Chantiers,CCTP_CSPS_Tranche 2 définitive.pdf
1,Cyrille CHARTIER - CYRILLE,01-CHANTIERS\02-EN ATTENTE\MONTOIR DE BZH - MA...,,MONTOIR DE BZH,MAIRIE,Ecole Jaurès & multi accueil,Chantiers,EJJ-MPE CSPS CCAP.pdf
2,Cyrille CHARTIER - CYRILLE,01-CHANTIERS\02-EN ATTENTE\MONTOIR DE BZH - MA...,,MONTOIR DE BZH,MAIRIE,Ecole Jaurès & multi accueil,Chantiers,EJJ-MPE CSPS CCTP.pdf
3,Cyrille CHARTIER - CYRILLE,01-CHANTIERS\02-EN ATTENTE\MONTOIR DE BZH - MA...,,MONTOIR DE BZH,MAIRIE,Ecole Jaurès & multi accueil,Chantiers,EJJ-MPE CSPS RC.pdf
4,Cyrille CHARTIER - CYRILLE,01-CHANTIERS\03-EN ATTENTE PHASE 2\CHAUMES EN ...,,CHAUMES EN RETZ,PORNIC AGGLO,Tx Alvéoles ISDND Écocentre Ste Anne,Chantiers,2023_300_CCTP_SPS_Travaux_alveoles_ISDND_Ecoce...


In [135]:
df_chantier.tail()

,SPS Name,Directory,ID EBP,Ville,Entreprise,Mission,statut,filename
1856,Yann HERVE - YANN,01-CHANTIERS\04-EN COURS\HERBIERS - COMCOM - B...,,HERBIERS,COMCOM,Bibliothèque,Chantiers,4-CCTP-Lot2CSPS.pdf
1857,Yann HERVE - YANN,01-CHANTIERS\04-EN COURS\ST FULGENT - MAIRIE &...,,ST FULGENT,MAIRIE & VE,QH La Crèche Auneau,Chantiers,CCAP.pdf
1858,Yann HERVE - YANN,01-CHANTIERS\05-DIUO A FAIRE\LIRE - DEPT49 - ...,,LIRE - DEPT49,Giratoire des vignes (09,24),Chantiers,CCAP.pdf
1859,Yann HERVE - YANN,01-CHANTIERS\05-DIUO A FAIRE\LIRE - DEPT49 - ...,,LIRE - DEPT49,Giratoire des vignes (09,24),Chantiers,CCTP.pdf
1860,Yann HERVE - YANN,01-CHANTIERS\05-DIUO A FAIRE\MONTAIGU (BOUFFER...,,MONTAIGU (BOUFFERE),SAS JESS,Agence VLOK,Chantiers,1 - CCTP V-LOK - BOUFFERE - 04.05.2022.pdf


In [136]:
df_chantier.info()

<class 'pandas.core.frame.DataFrame'>
Index: 573 entries, 0 to 1860
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   SPS Name    573 non-null    object
 1   Directory   573 non-null    object
 2   ID EBP      573 non-null    object
 3   Ville       573 non-null    object
 4   Entreprise  573 non-null    object
 5   Mission     573 non-null    object
 6   statut      573 non-null    object
 7   filename    573 non-null    object
dtypes: object(8)
memory usage: 40.3+ KB


In [137]:
df_EBP.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2961 entries, 0 to 2960
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   SPS Name    2961 non-null   object
 1   Directory   2961 non-null   object
 2   ID EBP      2961 non-null   object
 3   Ville       2961 non-null   object
 4   Entreprise  2961 non-null   object
 5   Mission     2961 non-null   object
 6   statut      2961 non-null   object
dtypes: object(7)
memory usage: 162.1+ KB


In [141]:

# Define the columns to match on
match_columns = ['SPS Name', 'Ville', 'Entreprise', 'Mission']

# Perform an inner merge on the specified columns
# An inner merge will only keep rows where the values in 'match_columns' are identical in both DataFrames.
matched_df = pd.merge(df_EBP, df_chantier, on=match_columns, how='inner', suffixes=('_ebp', '_chantier'))

matching_id_ebp_from_ebp = matched_df['ID EBP_ebp'].unique()

print("ID EBP from df_EBP that match in df_chantier based on specified columns:")
print(matching_id_ebp_from_ebp)

print(matched_df.head())

ID EBP from df_EBP that match in df_chantier based on specified columns:
[]
Empty DataFrame
Columns: [SPS Name, Directory_ebp, ID EBP_ebp, Ville, Entreprise, Mission, statut_ebp, Directory_chantier, ID EBP_chantier, statut_chantier, filename]
Index: []
